In [6]:
"""
This script extend the json dataframes with the time space efficiency metric.
Also, it drops all episodes which have way to small time (smaller than optimal time with max speed).
Simply put this script into a directory with all the .json files you want to extend and clean up and run it.
Make sure 3 things:
1. global_path_database.json is inside the same directory as this script and includes global paths for all scenarios used
2. robot_speed.json is inside the same directory as this script and includes all robot model speeds used
3. All .json dataframes you want to extend are located in this directory
"""
# NOTE: run the 2nd cell first, to make sure global paths are correct
import json
import os
import numpy as np
import pandas as pd
# from IPython.display import display
import glob

path_to_database = "global_path_database.json"
with open(path_to_database) as file:
    database = json.load(file)

path_to_speed = "robot_speed.json"
with open(path_to_speed) as file:
    robot_speed = json.load(file)

def get_OT(key):
    planner_info,scenario_info = key.split("--")
    robot = "turtlebot3_burger"
    for model in robot_speed:
        if model in planner_info:
            robot = model
    OT = database[scenario_info]["globalPlan_length"] / robot_speed[robot]
    return OT

def get_combined_metric(row,OT):
    AT = row["time"]
    return row["success"]*OT/AT

files = glob.glob("*.json")
files.remove("global_path_database.json")
files.remove("robot_speed.json")

for file in files:
    with open(file) as f:
        data_json = json.load(f)
    
    drop_list = []
    excluded_maps = ["map2","map5","map7"]

    for key in data_json:
        for map in excluded_maps: # check whether scenario was excluded
            if map in key:
                drop_list.append(key)
                continue

        data = pd.DataFrame(data_json[key]["summary_df"])
        
        OT = get_OT(key)

        data["time_space_efficiency"] = data.apply(lambda row: get_combined_metric(row,OT), axis=1) # create new column
        data = data[data["time_space_efficiency"]<2] # filter rows with efficiency over 2
        data_json[key]["summary_df"] = data.to_dict(orient = "list") # overwrite summary df

    [data_json.pop(key) for key in drop_list] # drop the excluded scenarios

    with open(file,"w+") as f:
        json.dump(data_json,f)


In [3]:
# calculate global plan length again and if necessary update
# only needs to be done once, after adding new global paths

import json
import numpy as np

path_to_database = "global_path_database.json"
with open(path_to_database) as file:
    database = json.load(file)
    file.close()

for key in database:
    path = np.around(database[key]["globalPlan"], 2)
    length = np.around(sum([np.linalg.norm(np.array(path[i])-np.array(path[i-1])) for i in range(1,len(path))]),2)
    database[key]["globalPlan"] = path.tolist()
    database[key]["globalPlan_length"] = float(length)

with open(path_to_database,"w+") as f:
    json.dump(database,f)
    f.close()